# 菜品分类--数据分组/数据透视表

## 数据分组

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'train-pivot.csv',encoding='utf-8')
df

,用户ID,客户分类,区域,是否省会,7月销量,8月销量,9月销量
0,59224,A类,一线城市,是,6,20,0
1,55295,B类,三线城市,否,37,27,35
2,46035,A类,二线城市,是,8,1,8
3,2459,C类,一线城市,是,7,8,14
4,22179,B类,三线城市,否,9,12,4


### 分组键是列名

#### 按照一列进行分组

`groupby()`方法获得一个 `DataFrameGroupBy object`

In [7]:
df.groupby('客户分类')

根据客户分类进行分组，然后对分组后的数据分别进行计数：

In [8]:
df.groupby('客户分类').count()

,用户ID,区域,是否省会,7月销量,8月销量,9月销量
客户分类,,,,,,
A类,2,2,2,2,2,2
B类,2,2,2,2,2,2
C类,1,1,1,1,1,1


对分组后的数据进行求和：

In [9]:
df.groupby('客户分类').sum()

,用户ID,7月销量,8月销量,9月销量
客户分类,,,,
A类,105259,14,21,8
B类,77474,46,39,39
C类,2459,7,8,14


`聚合`：对分组后的数据进行汇总运算的操作称为聚合，使用的函数称为`聚合函数`，8.3中学过的汇总运算函数都可以做为聚合函数对分组后的数据进行聚合。

【**重点**】以上都是对所有列进行计算，有时候我们不需要对所有的列进行汇总计算，这时可以把想要计算的列（可以是单列或多列）通过索引的方式取出来，然后在此基础上进行汇总计算:

In [15]:
df.groupby('客户分类')[['用户ID']].count()

,用户ID
客户分类,
A类,2
B类,2
C类,1


In [14]:
df.groupby('客户分类')[['用户ID','区域']].count()

,用户ID,区域
客户分类,,
A类,2,2
B类,2,2
C类,1,1


#### 按照多列进行分组

In [10]:
df.groupby(['客户分类','区域']).count()

用户ID  是否省会  7月销量  8月销量  9月销量
客户分类 区域                                
A类   一线城市     1     1     1     1     1
     二线城市     1     1     1     1     1
B类   三线城市     2     2     2     2     2
C类   一线城市     1     1     1     1     1

In [11]:
df.groupby(['客户分类','区域']).sum()

用户ID  7月销量  8月销量  9月销量
客户分类 区域                           
A类   一线城市  59224     6    20     0
     二线城市  46035     8     1     8
B类   三线城市  77474    46    39    39
C类   一线城市   2459     7     8    14

### 分组键是Series

`Series`：把`Dataframe`的其中一列取出来就是一个Series

给groupby()传入`Series`的效果和传入列名的效果是完全一样的

In [16]:
df.groupby(df['客户分类']).count()

,用户ID,区域,是否省会,7月销量,8月销量,9月销量
客户分类,,,,,,
A类,2,2,2,2,2,2
B类,2,2,2,2,2,2
C类,1,1,1,1,1,1


In [18]:
df.groupby([df['客户分类'],df['区域']]).sum()

用户ID  7月销量  8月销量  9月销量
客户分类 区域                           
A类   一线城市  59224     6    20     0
     二线城市  46035     8     1     8
B类   三线城市  77474    46    39    39
C类   一线城市   2459     7     8    14

In [19]:
df.groupby(df['客户分类'])[['用户ID','区域']].count()

,用户ID,区域
客户分类,,
A类,2,2
B类,2,2
C类,1,1


### 神奇的`aggregate`方法

#### 神奇之一：一次可以使用多做汇总方式

In [21]:
df.groupby('客户分类')[['用户ID','7月销量']].aggregate(['count','sum'])

用户ID          7月销量    
     count     sum count sum
客户分类                        
A类       2  105259     2  14
B类       2   77474     2  46
C类       1    2459     1   7

#### 神奇之二：可以针对不同的列做不同的汇总运算

In [23]:
df.groupby('客户分类')[['用户ID','7月销量']].aggregate({'用户ID':'count','7月销量':'sum'})

,用户ID,7月销量
客户分类,,
A类,2,14
B类,2,46
C类,1,7


### 对分组后的结果重置索引

In [25]:
df.groupby('客户分类').sum()

,用户ID,7月销量,8月销量,9月销量
客户分类,,,,
A类,105259,14,21,8
B类,77474,46,39,39
C类,2459,7,8,14


In [26]:
type(df.groupby('客户分类').sum())

pandas.core.frame.DataFrame

DataFrameGroupBy对象经过汇总运算之后成为DataFrame，但形式并不标准，利用重置索引reset_index()方法，可以把非标准形式转化为标准形式：

In [27]:
df.groupby('客户分类').sum().reset_index()

,客户分类,用户ID,7月销量,8月销量,9月销量
0,A类,105259,14,21,8
1,B类,77474,46,39,39
2,C类,2459,7,8,14


## 数据透视表

`pivot_table()`方法说明：

pd.pivot_table(  
    data,    --表示要做数据透视的整个表  
    values=None,    --对应Excel中值那个框  
    index=None,    --对应Excel中行那个框  
    columns=None,    --对应Excel中列那个框  
    aggfunc='mean',    --表示对values的计算类型  
    fill_value=None,    --表示对空值的填充值  
    margins=False,    --表示是否显示合计列  
    dropna=True,    --表示是否删除缺失，如果为真时，把一整行全做为缺失值删除  
    margins_name='All',    --表示合计列的列名  
)

最简单的基本使用：

In [28]:
pd.pivot_table(df,values='用户ID',columns='区域',index='客户分类',aggfunc='count')

区域,一线城市,三线城市,二线城市
客户分类,,,
A类,1.0,NaN,1.0
B类,NaN,2.0,NaN
C类,1.0,NaN,NaN


显示合计列：

In [29]:
pd.pivot_table(df,values='用户ID',columns='区域',index='客户分类',aggfunc='count',margins=True)

区域,一线城市,三线城市,二线城市,All
客户分类,,,,
A类,1.0,NaN,1.0,2
B类,NaN,2.0,NaN,2
C类,1.0,NaN,NaN,1
All,2.0,2.0,1.0,5


设置合计列的名称：

In [31]:
pd.pivot_table(df,values='用户ID',columns='区域',index='客户分类',aggfunc='count',margins=True,margins_name='总计')

区域,一线城市,三线城市,二线城市,总计
客户分类,,,,
A类,1.0,NaN,1.0,2
B类,NaN,2.0,NaN,2
C类,1.0,NaN,NaN,1
总计,2.0,2.0,1.0,5


填充缺失值为0

In [32]:
pd.pivot_table(df,values='用户ID',columns='区域',index='客户分类',aggfunc='count',margins=True,margins_name='总计',fill_value=0)

区域,一线城市,三线城市,二线城市,总计
客户分类,,,,
A类,1,0,1,2
B类,0,2,0,2
C类,1,0,0,1
总计,2,2,1,5


对不同的值进行不同类型的汇总计算，aggfunc参数传入一个字典

In [35]:
pd.pivot_table(df,values=['用户ID','7月销量'],columns='区域',index='客户分类',aggfunc={'用户ID':'count','7月销量':'sum'},fill_value=0)

7月销量           用户ID          
区域   一线城市 三线城市 二线城市 一线城市 三线城市 二线城市
客户分类                              
A类      6    0    8    1    0    1
B类      0   46    0    0    2    0
C类      7    0    0    1    0    0

`reset_index()`对透视结果重置索引

In [38]:
pd.pivot_table(df,values=['用户ID','7月销量'],columns='区域',index='客户分类',aggfunc={'用户ID':'count','7月销量':'sum'},fill_value=0).reset_index()

客户分类 7月销量           用户ID          
区域      一线城市 三线城市 二线城市 一线城市 三线城市 二线城市
0    A类    6    0    8    1    0    1
1    B类    0   46    0    0    2    0
2    C类    7    0    0    1    0    0

In [39]:
pd.pivot_table(df,values='用户ID',columns='区域',index='客户分类',aggfunc='count',margins=True,margins_name='总计',fill_value=0).reset_index()

区域,客户分类,一线城市,三线城市,二线城市,总计
0,A类,1,0,1,2
1,B类,0,2,0,2
2,C类,1,0,0,1
3,总计,2,2,1,5


思路更重要